### Part 1
A crop of this size requires significant logistics to transport produce, soil, fertilizer, and so on. The Elves are very busy pushing things around in carts on some kind of rudimentary system of tracks they've come up with.

Seeing as how cart-and-track systems don't appear in recorded history for another 1000 years, the Elves seem to be making this up as they go along. They haven't even figured out how to avoid collisions yet.

You map out the tracks (your puzzle input) and see where you can help.

Tracks consist of straight paths (| and -), curves (/ and \), and intersections (+). Curves connect exactly two perpendicular pieces of track; for example, this is a closed loop:
```
/----\
|    |
|    |
\----/
```
Intersections occur when two perpendicular paths cross. At an intersection, a cart is capable of turning left, turning right, or continuing straight. Here are two loops connected by two intersections:
```
/-----\
|     |
|  /--+--\
|  |  |  |
\--+--/  |
   |     |
   \-----/
```
Several carts are also on the tracks. Carts always face either up (^), down (v), left (<), or right (>). (On your initial map, the track under each cart is a straight path matching the direction the cart is facing.)

Each time a cart has the option to turn (by arriving at any intersection), it turns left the first time, goes straight the second time, turns right the third time, and then repeats those directions starting again with left the fourth time, straight the fifth time, and so on. This process is independent of the particular intersection at which the cart has arrived - that is, the cart has no per-intersection memory.

Carts all move at the same speed; they take turns moving a single step at a time. They do this based on their current location: carts on the top row move first (acting from left to right), then carts on the second row move (again from left to right), then carts on the third row, and so on. Once each cart has moved one step, the process repeats; each of these loops is called a tick.

For example, suppose there are two carts on a straight track:
```
|  |  |  |  |
v  |  |  |  |
|  v  v  |  |
|  |  |  v  X
|  |  ^  ^  |
^  ^  |  |  |
|  |  |  |  |
```
First, the top cart moves. It is facing down (v), so it moves down one square. Second, the bottom cart moves. It is facing up (^), so it moves up one square. Because all carts have moved, the first tick ends. Then, the process repeats, starting with the first cart. The first cart moves down, then the second cart moves up - right into the first cart, colliding with it! (The location of the crash is marked with an X.) This ends the second and last tick.

Here is a longer example:
```
/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \->--/
  \------/   

/---v        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   

/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   

/---\        
|   |  /----\
| /->--+-\  |
| | |  | |  |
\-+-/  \-+--^
  \------/   

/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   

/---\        
|   |  /----\
| /-+->+-\  ^
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /----<
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /---<\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /--<-\
| /-+--+-v  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /-<--\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /<---\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-<--/
  \------/   

/---\        
|   |  v----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \<+--/
  \------/   

/---\        
|   |  /----\
| /-+--v-\  |
| | |  | |  |
\-+-/  ^-+--/
  \------/   

/---\        
|   |  /----\
| /-+--+-\  |
| | |  X |  |
\-+-/  \-+--/
  \------/   
```
After following their respective paths for a while, the carts eventually crash. To help prevent crashes, you'd like to know the location of the first crash. Locations are given in X,Y coordinates, where the furthest left column is X=0 and the furthest top row is Y=0:
```
           111
 0123456789012
0/---\        
1|   |  /----\
2| /-+--+-\  |
3| | |  X |  |
4\-+-/  \-+--/
5  \------/   
```
In this example, the location of the first crash is 7,3.

In [1]:
from pathlib import Path
import itertools
from collections import Counter
from collections import defaultdict
from pprint import pprint
import re
from datetime import datetime
import numpy as np
import functools
from copy import copy, deepcopy

In [2]:
raw_data = Path('input.txt').read_text()

In [3]:
#left = 0
#straight = 1
#right = 2
#repeat...

mutations = {'}':{'^':'<', '>':'v', 'v':'>', '<':'^'},
             '/':{'^':'>', '<':'v', 'v':'<', '>':'^'},
             '+':{'^':{0:'<',1:'^',2:'>'},
                  '<':{0:'v',1:'<',2:'^'},
                  'v':{0:'>',1:'v',2:'<'},
                  '>':{0:'^',1:'>',2:'v'}},
             '-':{'<':'<', '>':'>'},
             '|':{'^':'^', 'v':'v'},
            '>':{'<':'X','v':'X','^':'X','>':'X'},
            'v':{'<':'X','v':'X','^':'X','>':'X'},
            '^':{'<':'X','v':'X','^':'X','>':'X'},
            '<':{'<':'X','v':'X','^':'X','>':'X'},}

In [4]:
carts = {'<':(-1, 0),
    '>':(1,0),
    'v':(0,1),
    '^':(0,-1)}

### Part 2
There isn't much you can do to prevent crashes in this ridiculous system. However, by predicting the crashes, the Elves know where to be in advance and instantly remove the two crashing carts the moment any crash occurs.

They can proceed like this for a while, but eventually, they're going to run out of carts. It could be useful to figure out where the last cart that hasn't crashed will end up.

For example:
```
/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/

/---\  
|   |  
| v-+-\
| | | |
\-+-/ |
  |   |
  ^---^

/---\  
|   |  
| /-+-\
| v | |
\-+-/ |
  ^   ^
  \---/

/---\  
|   |  
| /-+-\
| | | |
\-+-/ ^
  |   |
  \---/
```
After four very expensive crashes, a tick ends with only one cart remaining; its final location is 6,4.

What is the location of the last cart at the end of the first tick where it is the only cart left?

In [5]:
data = raw_data.replace('\\','}')
data_no_carts = deepcopy(data).replace('>','-').replace('<','-').replace('^','|').replace('v','|')

data = [list(row) for row in data.split('\n')]
data_no_carts = [list(row) for row in data_no_carts.split('\n')]

cart_tracking = {}

positions = []
new_positions = []

first_collision = False

cart_num = 0
for y, row in enumerate(data):
    for x, space in enumerate(row):
        if any(cart in space for cart in carts.keys()):
            cart_tracking[cart_num] = (x, y, space, 0)
            cart_num += 1
#for row in data:
#    print(''.join(row))

while len(cart_tracking) > 1:
    remove_ids = set()
    for cart_id, (x, y, space, num_turns) in sorted(cart_tracking.items(), key = lambda x: (x[1][1], x[1][0])): # go through carts in proper order
        if cart_id not in remove_ids:
            #print(y, x)
            x_adj, y_adj = carts[space]
            new_space = data[y+y_adj][x+x_adj]

            if new_space == '+':
                mutate = mutations[new_space][space][num_turns]
                num_turns += 1
            else:
                try:
                    mutate = mutations[new_space][space]
                except KeyError:
                    print(x, y, x_adj, y_adj, new_space, space)
                    for _r in data[0:7]:
                        print(''.join(_r[0:55]))
                    print()
                    for _r in data_no_carts[0:7]:
                        print(''.join(_r[0:55]))
                    raise KeyError

            if mutate == 'X':
                if not first_collision:
                    print((x+x_adj, y+y_adj))
                    first_collision = True

                #remove carts:
                remove_ids.add(cart_id)
                mutate = data_no_carts[y+y_adj][x+x_adj]
                for other_cart_id, (other_x, other_y, _, _) in cart_tracking.items():
                    if (other_x,other_y) == (x+x_adj, y+y_adj):
                        remove_ids.add(other_cart_id)


            cart_tracking[cart_id] = (x+x_adj, y+y_adj, mutate, num_turns % 3)
            data[y][x] = data_no_carts[y][x]
            data[y+y_adj][x+x_adj] = mutate

    for i in remove_ids:
        del cart_tracking[i]

    #for row in data:
    #    print(''.join(row))
print(tuple(list(cart_tracking.values())[0][0:2]))

(91, 69)
(44, 87)
